**This Jupyter notebook for:**
- reading clustered building point cloud (RGB-coded clusters) from a .laz/.las/.ply
- building 2D footprints per cluster (concave hull / alpha shape)
- estimating height for each building (from original z values)
- extruding footprints to meshes with random (or data-derived) heights
- merging all building meshes into a single mesh and exporting (GLB/OBJ/PLY)

In [1]:
# CELL 1 — Install prerequisites (run in your environment; comment out if already installed)
# ----------------------------------
# !pip install laspy open3d shapely alphashape trimesh pygltflib numpy scipy


# Note: if you prefer PDAL or have huge files, use PDAL to preprocess and/or read LAZ.
# If internet access for pip is not available in your environment, install packages manually.

In [2]:
# CELL 2 — Imports & helper functions
# ----------------------------------
import os
import json
import math
import random
import numpy as np

# Prefer laspy for LAS/LAZ, fallback to open3d for PLY
try:
    import laspy
    LASPY_AVAILABLE = True
except Exception:
    LASPY_AVAILABLE = False

try:
    import open3d as o3d
    O3D_AVAILABLE = True
except Exception:
    O3D_AVAILABLE = False

from shapely.geometry import MultiPoint, Polygon
import alphashape
import trimesh

print('Dependencies: laspy:', LASPY_AVAILABLE, 'open3d:', O3D_AVAILABLE)

# Utility: read points + colors from file
def read_point_cloud(path):
    """Return numpy arrays: points (N x 3), colors (N x 3) in 0-255 ints or None."""
    ext = os.path.splitext(path)[1].lower()
    if ext in ('.las', '.laz'):
        if not LASPY_AVAILABLE:
            raise RuntimeError('laspy not available, install it or use PLY/PLY format')
        las = laspy.read(path)
        x = las.x
        y = las.y
        z = las.z
        points = np.vstack((x, y, z)).T
        # Colors: many LAS use red/green/blue fields or RGB as 16-bit; normalize if present
        colors = None
        if 'red' in las.point_format.standard_dimension_names or 'rgb' in las.point_format.dimension_names:
            try:
                r = las.red if 'red' in las.point_format.dimension_names else las['red']
                g = las.green if 'green' in las.point_format.dimension_names else las['green']
                b = las.blue if 'blue' in las.point_format.dimension_names else las['blue']
                # convert to 0-255 if in 16-bit
                if r.max() > 255:
                    r = (r / 256).astype(np.uint8)
                    g = (g / 256).astype(np.uint8)
                    b = (b / 256).astype(np.uint8)
                colors = np.vstack((r, g, b)).T.astype(np.uint8)
            except Exception:
                colors = None
        return points, colors

    elif ext in ('.ply',):
        if not O3D_AVAILABLE:
            raise RuntimeError('open3d not available to read PLY')
        pc = o3d.io.read_point_cloud(path)
        pts = np.asarray(pc.points)
        cols = None
        if pc.has_colors():
            cols = (np.asarray(pc.colors) * 255).astype(np.uint8)
        return pts, cols
    else:
        raise ValueError('Unsupported file type: ' + ext)

# Utility: compute concave (alpha) hull from 2D points
def footprint_from_points_xy(points_xy, alpha=None, min_area=0.1):
    """Return shapely Polygon or None.
    points_xy: (N,2) numpy array
    alpha: None=auto, or float. If auto, we compute an alpha by area heuristic.
    """
    if len(points_xy) < 4:
        return None
    try:
        if alpha is None:
            # auto alpha: use bounding box scale heuristic
            bbox = MultiPoint(points_xy).convex_hull.bounds
            bbox_diag = math.hypot(bbox[2]-bbox[0], bbox[3]-bbox[1])
            alpha = max(0.5, bbox_diag / 50.0)
        hull = alphashape.alphashape(points_xy, alpha)
        if not isinstance(hull, Polygon):
            hull = hull.convex_hull
        if hull.area < min_area:
            return None
        return hull
    except Exception as e:
        # fallback to convex hull
        return MultiPoint(points_xy).convex_hull

# Utility: extrude polygon to trimesh at given height
def extrude_polygon_to_mesh(polygon: Polygon, height: float, color=(200,100,50)):
    # shapely polygon -> trimesh extrude
    mesh = trimesh.creation.extrude_polygon(polygon, height)
    # assign uniform color
    verts = len(mesh.vertices)
    mesh.visual.vertex_colors = np.tile(np.array(color + (255,)), (verts,1))
    return mesh

# Utility: estimate building height from original 3D points for cluster
def estimate_height(z_vals):
    # robust estimate using percentile range, ignoring outliers
    h = np.percentile(z_vals, 95) - np.percentile(z_vals, 5)
    if h <= 0: h = np.ptp(z_vals)
    if math.isnan(h) or h <= 0: h = 10.0
    return h

# End of helper cell
print('Helper functions ready')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Dependencies: laspy: True open3d: True
Helper functions ready


In [3]:
# CELL 3 — Parameters (edit these for your dataset)
# -----------------------------------------------
INPUT_PATH = './input_buildings/buildings_colored.laz'   # path to your clustered, flattened file (PLY or LAS/LAZ)
OUTPUT_DIR = 'output_buildings'        # where to write individual glb or merged glb
EXPORT_MERGED = True                   # if True export merged single glb, else export individual glbs
MIN_POINTS_PER_CLUSTER = 20            # ignore tiny clusters/noise
ALPHA = None                           # control concave hull; None=auto
EXTRUDE_HEIGHT_MIN = 6.0
EXTRUDE_HEIGHT_MAX = 20.0

os.makedirs(OUTPUT_DIR, exist_ok=True)
print('Parameters set. Input:', INPUT_PATH, 'Output dir:', OUTPUT_DIR)

Parameters set. Input: ./input_buildings/buildings_colored.laz Output dir: output_buildings


In [4]:
# CELL 4 — Load point cloud
# -------------------------
points, colors = read_point_cloud(INPUT_PATH)
print('Loaded points:', points.shape)
if colors is None:
    raise RuntimeError('No colors found. Clusters must be color-coded.')
print('Loaded colors:', colors.shape)

Loaded points: (7226818, 3)
Loaded colors: (7226818, 3)


In [5]:
# CELL 5 — Group points by color and build footprints
# ---------------------------------------------------
# Convert colors array to tuple keys
color_tuples = [tuple(c) for c in colors]
from collections import defaultdict
idx_by_color = defaultdict(list)
for i, col in enumerate(color_tuples):
    idx_by_color[col].append(i)

print('Found', len(idx_by_color), 'unique colors (clusters)')

meshes = []
metadata = []
cluster_count = 0
for col, idxs in idx_by_color.items():
    if len(idxs) < MIN_POINTS_PER_CLUSTER:
        continue
    cluster_pts = points[idxs]
    # flatten XY footprint
    xy = cluster_pts[:, :2]
    hull = footprint_from_points_xy(xy, alpha=ALPHA)
    if hull is None:
        continue
    # estimate height from original z values
    zvals = cluster_pts[:, 2]
    height = estimate_height(zvals)
    # optionally randomize height
    height = random.uniform(EXTRUDE_HEIGHT_MIN, EXTRUDE_HEIGHT_MAX)

    mesh = extrude_polygon_to_mesh(hull, height, color=col)
    meshes.append(mesh)
    metadata.append({
        'color': col,
        'n_points': len(idxs),
        'height': float(height),
        'area': float(hull.area)
    })
    cluster_count += 1

print('Built', cluster_count, 'building meshes')

Found 352 unique colors (clusters)


Built 351 building meshes


In [7]:
# CELL 6 — Export: individual or merged
# --------------------------------------
if EXPORT_MERGED:
    merged = trimesh.util.concatenate(meshes)
    out_path = os.path.join(OUTPUT_DIR, 'city_merged.glb')
    merged.export(out_path)
    print('Exported merged mesh to', out_path)
else:
    for i, m in enumerate(meshes):
        out_path = os.path.join(OUTPUT_DIR, f'building_{i}.glb')
        m.export(out_path)
    print('Exported', len(meshes), 'individual building files to', OUTPUT_DIR)

# # metadata export
# meta_path = os.path.join(OUTPUT_DIR, 'buildings_metadata.json')
# with open(meta_path, 'w') as f:
#     json.dump(metadata, f, indent=2)
# print('Saved metadata to', meta_path)

# Final: summary
print('Done. Generated', len(meshes), 'building meshes.')
print('You can now load the generated glb(s) into Three.js or Potree viewer.')

Exported merged mesh to output_buildings\city_merged.glb
Done. Generated 351 building meshes.
You can now load the generated glb(s) into Three.js or Potree viewer.
